In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import itertools
import statsmodels.api as sm # 통계분석 기능을 제공하는 파이썬 패키지
import matplotlib
import itertools # 반복가능한 데이터 스트림을 처리하는데 유용한 많은 함수와 제네레이터 포함
from datetime import datetime
from pandas import DataFrame, Series
# 차트 기본 크기 설정
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [2]:
pd.set_option('display.max_columns', 500) # 생략없이 출력

In [3]:
df = pd.read_csv('C:/Users/JI/Desktop/LaLiga_dataset/all_season.csv')
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df[(((df['HomeTeam'] == 'Real Madrid') & (df['FTR'] == 'H'))
        |((df['AwayTeam'] == 'Real Madrid') & (df['FTR'] == 'A')))
        |(((df['HomeTeam'] == 'Real Madrid')|(df['AwayTeam'] == 'Real Madrid'))& (df['FTR'] == 'D'))
        |(((df['HomeTeam'] == 'Real Madrid') & (df['FTR'] == 'A'))
        |((df['AwayTeam'] == 'Real Madrid') & (df['FTR'] == 'H')))]

# 데이터 개수랑 맨 앞 숫자가 다르므로 data index가 바뀐 것
# index가 바꼈으니까 reset_index 등으로 인덱스 리셋

print(df.shape) # dropna 이전 데이터 쉐입 체크필요
df = df.sort_values('Date')
df = df.dropna(axis=1)
print(df.shape) # dropna 이후 데이터 쉐입 체크필요
df.head()

(996, 95)
(996, 7)


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
7224,SP1,1993-09-05,Osasuna,Real Madrid,1,4,A
7234,SP1,1993-09-12,Real Madrid,Valladolid,1,3,A
7242,SP1,1993-09-19,La Coruna,Real Madrid,4,0,H
7254,SP1,1993-09-26,Real Madrid,Oviedo,0,1,A
7261,SP1,1993-10-03,Ath Madrid,Real Madrid,0,0,D


In [4]:
# reset_index로 index 변화 체크
df = df.reset_index(drop=True)
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,SP1,1993-09-05,Osasuna,Real Madrid,1,4,A
1,SP1,1993-09-12,Real Madrid,Valladolid,1,3,A
2,SP1,1993-09-19,La Coruna,Real Madrid,4,0,H
3,SP1,1993-09-26,Real Madrid,Oviedo,0,1,A
4,SP1,1993-10-03,Ath Madrid,Real Madrid,0,0,D


In [5]:
df['Weekday'] = df['Date'].dt.day_name()
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Weekday
0,SP1,1993-09-05,Osasuna,Real Madrid,1,4,A,Sunday
1,SP1,1993-09-12,Real Madrid,Valladolid,1,3,A,Sunday
2,SP1,1993-09-19,La Coruna,Real Madrid,4,0,H,Sunday
3,SP1,1993-09-26,Real Madrid,Oviedo,0,1,A,Sunday
4,SP1,1993-10-03,Ath Madrid,Real Madrid,0,0,D,Sunday


In [6]:
# get_dummies하면 columns명이 위의 value
one_hot_encoded = pd.get_dummies(df['Weekday'])
print(one_hot_encoded.shape)
one_hot_encoded.head()

(996, 7)


,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0


In [7]:
# 좌우로 컨캣하면 데이터 개수가 996개로 같으므로 문제없음
df = pd.concat([df, pd.get_dummies(df['Weekday'])], axis=1)
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,SP1,1993-09-05,Osasuna,Real Madrid,1,4,A,Sunday,0,0,0,1,0,0,0
1,SP1,1993-09-12,Real Madrid,Valladolid,1,3,A,Sunday,0,0,0,1,0,0,0
2,SP1,1993-09-19,La Coruna,Real Madrid,4,0,H,Sunday,0,0,0,1,0,0,0
3,SP1,1993-09-26,Real Madrid,Oviedo,0,1,A,Sunday,0,0,0,1,0,0,0
4,SP1,1993-10-03,Ath Madrid,Real Madrid,0,0,D,Sunday,0,0,0,1,0,0,0


In [8]:
def win(x):
    if x ['HomeTeam']=='Real Madrid' and x['FTR'] == 'H':
        return 1 
    elif x['AwayTeam']=='Real Madrid' and x['FTR'] =='A':
        return 1
    else:
        return 0
df['FTR'] = df.apply(win, axis=1)
print(df)

     Div       Date     HomeTeam     AwayTeam  FTHG  FTAG  FTR   Weekday  \
0    SP1 1993-09-05      Osasuna  Real Madrid     1     4    1    Sunday   
1    SP1 1993-09-12  Real Madrid   Valladolid     1     3    0    Sunday   
2    SP1 1993-09-19    La Coruna  Real Madrid     4     0    0    Sunday   
3    SP1 1993-09-26  Real Madrid       Oviedo     0     1    0    Sunday   
4    SP1 1993-10-03   Ath Madrid  Real Madrid     0     0    0    Sunday   
..   ...        ...          ...          ...   ...   ...  ...       ...   
991  SP1 2019-04-25       Getafe  Real Madrid     0     0    0  Thursday   
992  SP1 2019-04-28    Vallecano  Real Madrid     1     0    0    Sunday   
993  SP1 2019-05-05  Real Madrid   Villarreal     3     2    1    Sunday   
994  SP1 2019-05-12     Sociedad  Real Madrid     3     1    0    Sunday   
995  SP1 2019-05-19  Real Madrid        Betis     0     2    0    Sunday   

     Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday  
0         0      

In [9]:
df = df[['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
         'Weekday', 'Monday', 'Tuesday','Wednesday','Thursday', 'Friday',
         'Saturday','Sunday']]
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Weekday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,SP1,1993-09-05,Osasuna,Real Madrid,1,4,1,Sunday,0,0,0,0,0,0,1
1,SP1,1993-09-12,Real Madrid,Valladolid,1,3,0,Sunday,0,0,0,0,0,0,1
2,SP1,1993-09-19,La Coruna,Real Madrid,4,0,0,Sunday,0,0,0,0,0,0,1
3,SP1,1993-09-26,Real Madrid,Oviedo,0,1,0,Sunday,0,0,0,0,0,0,1
4,SP1,1993-10-03,Ath Madrid,Real Madrid,0,0,0,Sunday,0,0,0,0,0,0,1


In [10]:
from sklearn.model_selection import train_test_split
X = df[['FTHG', 'FTAG','Monday','Tuesday','Wednesday','Thursday', 'Friday','Saturday','Sunday']]
y = df[['FTR']]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=100)


from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train, y_train)
rf_model.score(X_train, y_train)
y_pred = rf_model.predict(X_test)


<ipython-input-10-8e29f99002b6>:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model = rf.fit(X_train, y_train)


In [11]:
y_pred

array([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1], dtyp

In [12]:
round(rf.score(X_test, y_test), 4)

0.7659

In [13]:
sum(y_pred) / len(y_pred)

0.7591973244147158

In [21]:
df_20 = pd.read_csv('C:/Users/JI/Desktop/LaLiga_dataset/all_season.csv')
df_20['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df_20 = df_20[(((df_20['HomeTeam'] == 'Real Madrid') & (df_20['FTR'] == 'H'))
        |((df_20['AwayTeam'] == 'Real Madrid') & (df_20['FTR'] == 'A')))
        |(((df_20['HomeTeam'] == 'Real Madrid')|(df_20['AwayTeam'] == 'Real Madrid'))& (df_20['FTR'] == 'D'))
        |(((df_20['HomeTeam'] == 'Real Madrid') & (df_20['FTR'] == 'A'))
        |((df_20['AwayTeam'] == 'Real Madrid') & (df_20['FTR'] == 'H')))]
print(df_20.shape) # dropna 이전 데이터 쉐입 체크필요
df_20 = df_20.sort_values('Date')
df_20 = df_20.dropna(axis=1)
print(df_20.shape) # dropna 이후 데이터 쉐입 체크필요
# reset_index로 index 변화 체크
df_20 = df_20.reset_index(drop=True)
df_20.head(40)

(996, 95)
(996, 6)


,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,SP1,Real Madrid,Valencia,2,1,H
1,SP1,Malaga,Real Madrid,3,3,D
2,SP1,Real Madrid,Ath Bilbao,4,1,H
3,SP1,Santander,Real Madrid,0,0,D
4,SP1,Real Madrid,La Coruna,3,0,H
5,SP1,Barcelona,Real Madrid,2,0,H
6,SP1,Sociedad,Real Madrid,1,4,A
7,SP1,Real Madrid,Mallorca,0,2,A
8,SP1,Numancia,Real Madrid,3,1,H
9,SP1,Real Madrid,Las Palmas,5,1,H


In [18]:
def win(x):
    if x ['HomeTeam']=='Real Madrid' and x['FTR'] == 'H':
        return 1 
    elif x['AwayTeam']=='Real Madrid' and x['FTR'] =='A':
        return 1
    else:
        return 0
df_20['FTR'] = df_20.apply(win, axis=1)
print(df_20)

     Div     HomeTeam     AwayTeam  FTHG  FTAG  FTR
0    SP1  Real Madrid     Valencia     2     1    1
1    SP1       Malaga  Real Madrid     3     3    0
2    SP1  Real Madrid   Ath Bilbao     4     1    1
3    SP1    Santander  Real Madrid     0     0    0
4    SP1  Real Madrid    La Coruna     3     0    1
..   ...          ...          ...   ...   ...  ...
991  SP1  Real Madrid    Santander     2     4    0
992  SP1      Espanol  Real Madrid     0     2    1
993  SP1  Real Madrid       Alaves     0     1    0
994  SP1        Betis  Real Madrid     0     2    1
995  SP1  Real Madrid   Valladolid     0     1    0

[996 rows x 6 columns]


In [19]:
sum(df_20['FTR']) / len(df_20['FTR'])

0.6265060240963856